<a href="https://colab.research.google.com/github/Salman1450/CXR-and-MRI/blob/main/Inceptionv3_Unet_Segmentation_and_Dice_coef_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q segmentation-models
!pip install -q tensorflow==2.5
!pip install -q keras==2.2.4
# !pip install -q tensorflow-estimator==2.2.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.4/454.4 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 16.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 92.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.4/462.4 KB 49.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray 2022.12.0 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
xarray-einsta

In [ ]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

from tensorflow import keras
import segmentation_models as sm

Segmentation Models: using `tf.keras` framework.


In [ ]:
!pip install --upgrade --no-cache-dir gdown
!gdown --id 1CWP7QYdX-Y67FrFF0XG36SThtIOX7-dC
!pip install patool
import patoolib
patoolib.extract_archive('Tumor_seg.zip')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0
/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1CWP7QYdX-Y67FrFF0XG36SThtIOX7-dC
To: /content/Tumor_seg.zip
100% 332M/332M [00:01<00:00, 202MB/s]
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 KB 7.0 MB/s eta 0:00:00
patool: Extracting Tumor_seg.zip ...
patool: running /usr/bin/7z x -o./Unpack_9_u8ljv6 -- Tumor_seg.zip
patool: ... Tumor_seg.zip extracted to `Segmentation'.


'Segmentation'

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Reshape,Input
from tensorflow import float32


def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import Recall, Precision

""" Global parameters """
H = 256
W = 256

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_dataset(path, split=0.2):
    images = sorted(glob(os.path.join(path, "images", "*.png")))
    masks = sorted(glob(os.path.join(path, "masks", "*.png")))
    
    split_size = int(len(images) * split)
    
    train_x, valid_x = train_test_split(images, test_size=split_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=split_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=split_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)
def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    x = x / 255.0
    x = x.astype(np.float32)
    return x
def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (h, w)
    x = cv2.resize(x, (W, H))   ## (h, w)
    x = x / 255.0               ## (h, w)
    x = x.astype(np.float32)    ## (h, w)
    x = np.expand_dims(x, axis=-1)## (h, w, 1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset

In [ ]:
if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory for storing files """
    create_dir("files")

    """ Hyperparameters """
    batch_size = 16
    lr = 1e-4
    num_epochs = 30
    model_path = os.path.join("files", "inception_Unet_model.h5")
    csv_path = os.path.join("files", "inception_UNET_data.csv")

    """ Dataset """
    dataset_path = "/content/Segmentation"
    images = sorted(glob(os.path.join(dataset_path, "images", "*.png")))
    print(type(images))
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_dataset(dataset_path)
    print(type(train_x))
    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Valid: {len(valid_x)} - {len(valid_y)}")
    print(f"Test : {len(test_x)} - {len(test_y)}")

    train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

    """ Model """
    BACKBONE = 'inceptionv3'
    model = sm.Unet(BACKBONE, encoder_weights='imagenet')
    metrics = [dice_coef, iou, Recall(), Precision()]
    model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=["accuracy",metrics])
    model.summary()

<class 'list'>
<class 'list'>
Train: 1852 - 1852
Valid: 616 - 616
Test : 616 - 616
87916544/87910968 [==============================] - 9s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, None, 3 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, None, 

In [ ]:
callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path),
        EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=False),
    ]

model.fit(
        train_dataset,
        epochs=num_epochs,
        validation_data=valid_dataset,
        callbacks=callbacks
    )

Epoch 1/30
116/116 [==============================] - 77s 422ms/step - loss: 0.8926 - accuracy: 0.8554 - dice_coef: 0.1075 - iou: 0.0573 - recall: 0.9242 - precision: 0.1044 - val_loss: 0.9299 - val_accuracy: 0.7330 - val_dice_coef: 0.0702 - val_iou: 0.0364 - val_recall: 0.9822 - val_precision: 0.0571

Epoch 00001: val_loss improved from inf to 0.92989, saving model to files/inception_Unet_model.h5
Epoch 2/30
116/116 [==============================] - 42s 359ms/step - loss: 0.7088 - accuracy: 0.9799 - dice_coef: 0.2914 - iou: 0.1732 - recall: 0.9036 - precision: 0.4817 - val_loss: 0.7679 - val_accuracy: 0.9634 - val_dice_coef: 0.2322 - val_iou: 0.1317 - val_recall: 0.9570 - val_precision: 0.3102

Epoch 00002: val_loss improved from 0.92989 to 0.76789, saving model to files/inception_Unet_model.h5
Epoch 3/30
116/116 [==============================] - 41s 357ms/step - loss: 0.4589 - accuracy: 0.9909 - dice_coef: 0.5412 - iou: 0.3748 - recall: 0.8757 - precision: 0.7192 - val_loss: 0.4996

In [ ]:
patoolib.extract_archive('SOMC_Seg.zip')

patool: Extracting SOMC_Seg.zip ...
patool: running /usr/bin/7z x -o./Unpack_7hs9spw8 -- SOMC_Seg.zip
patool: ... SOMC_Seg.zip extracted to `SOMC_Seg'.


'SOMC_Seg'

In [ ]:
import shutil
shutil.copy("/content/files/inception_Unet_model.h5","/content/drive/MyDrive/Tumor")

'/content/drive/MyDrive/Tumor/inception_Unet_model.h5'

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
 
if __name__ == "__main__":
 
    """ Loading model """
    with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}):
        model = tf.keras.models.load_model("/content/files/inception_Unet_model.h5")

    """ Dataset """
    path = "/content/SOMC_Seg/MRI_SOMC"
    test=sorted(glob(os.path.join(path, "*.png")))
    create_dir("Output")

    """ Predicting the mask """
    for x in tqdm( test, total=len(test )):
        """ Extracing the image name. """
        image_name = x.split("/")[-1]

        """ Reading the image """
        img = cv2.imread(x, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (W, H))
        x = img/255.0
        x = x.astype(np.float32)
        x = np.expand_dims(x, axis=0)
        """ Predicting the mask. """
        y_pred = model.predict(x)[0] > 0.5
        print(type(y_pred))
        y_pred = y_pred.astype(np.int32)

        """ Saving the predicted mask along with the image and GT """
        save_image_path = f"/content/Output/{image_name}"
        y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)


        output_image = np.concatenate([y_pred*255], axis=1)
        cv2.imwrite(save_image_path, output_image)

 10%|█         | 2/20 [00:02<00:15,  1.18it/s]

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


 30%|███       | 6/20 [00:02<00:03,  4.31it/s]

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


 40%|████      | 8/20 [00:02<00:02,  5.85it/s]

...

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


 90%|█████████ | 18/20 [00:03<00:00, 10.77it/s]

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


100%|██████████| 20/20 [00:03<00:00,  5.72it/s]

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [ ]:
!zip -r /content/Output.zip  /content/Output

  adding: content/Output/ (stored 0%)
  adding: content/Output/3069.png (deflated 60%)
  adding: content/Output/3077.png (deflated 53%)
  adding: content/Output/3068.png (deflated 58%)
  adding: content/Output/3082.png (deflated 44%)
  adding: content/Output/3084.png (deflated 48%)
  adding: content/Output/3067.png (deflated 57%)
  adding: content/Output/3083.png (deflated 47%)
  adding: content/Output/3075.png (deflated 51%)
  adding: content/Output/3080.png (deflated 45%)
  adding: content/Output/3074.png (deflated 58%)
  adding: content/Output/3081.png (deflated 43%)
  adding: content/Output/3076.png (deflated 47%)
  adding: content/Output/3079.png (deflated 42%)
  adding: content/Output/3071.png (deflated 49%)
  adding: content/Output/3072.png (deflated 44%)
  adding: content/Output/3073.png (deflated 43%)
  adding: content/Output/3066.png (deflated 60%)
  adding: content/Output/3078.png (deflated 50%)
  adding: content/Output/3065.png (deflated 56%)
  adding: content/Output/3070.p

In [ ]:
import shutil
shutil.copy("/content/Output.zip","/content/drive/MyDrive/Tumor")

'/content/drive/MyDrive/Tumor/Output.zip'

In [ ]:
shutil.copy("/content/files/inception_UNET_data.csv","/content/drive/MyDrive/Tumor")

'/content/drive/MyDrive/Tumor/inception_UNET_data.csv'

In [ ]:
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)


path1 = "/content/SOMC_Seg/Mask_SOMC"
path2 = "/content/Output"
Original=sorted(glob(os.path.join(path1, "*.png")))
Output=sorted(glob(os.path.join(path2, "*.png")))

for x, y  in tqdm(zip(Original,Output), total=len(Output)):
    print("\n")
    image_name = x.split("/")[-1]
    image_name2 = y.split("/")[-1]
    x = cv2.imread(x, cv2.IMREAD_GRAYSCALE)  ## (h, w)
    x = cv2.resize(x, (W, H))   ## (h, w)
    x = x / 255.0               ## (h, w)
    x = x.astype(np.float32)    ## (h, w)
    x = np.expand_dims(x, axis=-1)## (h, w, 1)
    
    y = cv2.imread(y, cv2.IMREAD_GRAYSCALE)  ## (h, w)
    y = cv2.resize(y, (W, H))   ## (h, w)
    y = y / 255.0               ## (h, w)
    y = y.astype(np.float32)    ## (h, w)
    y = np.expand_dims(y, axis=-1)## (h, w, 1)
    

    print(f"For image {image_name} and image {image_name2} the Dice Coef is = {dice_coef(x,y)}")
    print(f"For image {image_name} and image {image_name2} the IOU is = {iou(x,y)}")



100%|██████████| 20/20 [00:00<00:00, 156.53it/s]



For image 3065.png and image 3065.png the Dice Coef is = 3.497865569078486e-19
For image 3065.png and image 3065.png the IOU is = 3.497865569078486e-19


For image 3066.png and image 3066.png the Dice Coef is = 0.9702081680297852
For image 3066.png and image 3066.png the IOU is = 0.9421399235725403


For image 3067.png and image 3067.png the Dice Coef is = 0.9141136407852173
For image 3067.png and image 3067.png the IOU is = 0.8418133854866028


For image 3068.png and image 3068.png the Dice Coef is = 0.9186642169952393
For image 3068.png and image 3068.png the IOU is = 0.8495641350746155


For image 3069.png and image 3069.png the Dice Coef is = 0.9655382633209229
For image 3069.png and image 3069.png the IOU is = 0.9333727955818176


For image 3070.png and image 3070.png the Dice Coef is = 0.9625996351242065
For image 3070.png and image 3070.png the IOU is = 0.9278960227966309


For image 3071.png and image 3071.png the Dice Coef is = 0.9783216714859009
For image 3071.png and image